In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Paper: [Training Generative Adversarial Networks with Limited Data](https://arxiv.org/abs/2006.06676)

![](https://github.com/NVlabs/stylegan2-ada-pytorch/raw/main/docs/stylegan2-ada-teaser-1024x252.png)

## Repro [Github](https://github.com/NVlabs/stylegan2-ada-pytorch)

In [2]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.1/196.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 36.2 MB/s eta 0:00:00
  Attempting uninstall: imageio-ffmpeg
    Found existing installation: imageio-ffmpeg 0.6.0
    Uninstalling imageio-ffmpeg-0.6.0:
      Successfully uninstalled imageio-ffmpeg-0.6.0


In [3]:
# add align_images.py
# !git clone https://github.com/rkuo2000/stylegan2-ada-pytorch
!git clone https://github.com/NVlabs/stylegan3
%cd stylegan3

Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 212 (delta 99), reused 90 (delta 90), pack-reused 49 (from 1)
Receiving objects: 100% (212/212), 4.16 MiB | 7.42 MiB/s, done.
Resolving deltas: 100% (108/108), done.
/content/stylegan3


## Training new networks

In [4]:
!python dataset_tool.py --source=/content/drive/MyDrive/Estudo/PUC/INCA/max_constriction/ --dest=../data.zip

100% 100/100 [00:09<00:00, 10.92it/s]


In [5]:
# !python train.py --outdir=~/training-runs --data=../data.zip --cfg=stylegan2 --gpus=1 --batch=16 --gamma=0.8192 --map-depth=2 --glr=0.0025 --dlr=0.0025 --cbase=16384

!python train.py \
  --outdir=../training-runs \
  --data=../data.zip \
  --cfg=stylegan2 \
  --gpus=1 \
  --batch=16 \
  --gamma=0.8192 \
  --map-depth=2 \
  --glr=0.0025 \
  --dlr=0.0025 \
  --cbase=16384 \
  --kimg=1000 \
  --metrics=none \
  # --tick=1 \
  --snap=5

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'

Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan2.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 16384,
    "channel_max": 512,
    "fused_modconv_default": "inference_only"
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 0.8192,
    "st

## Generate
Pre-trained networks are stored as *.pkl files that can be referenced using local filenames or URLs:

* Generate curated MetFaces images without truncation (Fig.10 left)

    python generate.py --outdir=out --trunc=1 --seeds=85,265,297,849 \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

* Generate uncurated MetFaces images with truncation (Fig.12 upper left)

    python generate.py --outdir=out --trunc=0.7 --seeds=600-605 \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

* Generate class conditional CIFAR-10 images (Fig.17 left, Car)

    python generate.py --outdir=out --seeds=0-35 --class=1 \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/cifar10.pkl

* Style mixing example

    python style_mixing.py --outdir=out --rows=85,100,458,1500 --cols=55,821,1789,293 \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

In [ ]:
!ls -l ~/

In [ ]:
# Generate curated MetFaces image without truncation
!python generate.py --outdir=out --trunc=1 --seeds=85,265,297,849 \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

In [ ]:
import os
import matplotlib.pyplot as plt

# display foler's images in 2x2
def plot_images(img_dir, top=2):
    all_img_dirs = os.listdir(img_dir)
    img_files = [os.path.join(img_dir, file) for file in all_img_dirs]

    plt.figure(figsize=(10, 10))

    for idx, img_path in enumerate(img_files):
        plt.subplot(2, 2, idx+1)

        img = plt.imread(img_path)
        plt.tight_layout()
        plt.imshow(img, cmap='gray')
        plt.axis('off')

In [ ]:
# display images in output folder
plot_images('./out')

In [ ]:
# Generate curated MetFaces image without truncation
!python generate.py --outdir=out --trunc=1 --seeds=85,265,297,849 \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

In [ ]:
plot_images('./out')

In [ ]:
!rm out/*

In [ ]:
# Style mixing example
!python style_mixing.py --outdir=out --rows=85,100,458,1500 --cols=55,821,1789,293 --styles=0-6  \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

In [ ]:
!ls out

In [ ]:
# display images in a row
def plot_pics(img_files):
    plt.figure(figsize=(10, 10))

    for i in range(len(img_files)):
        plt.subplot(1, len(img_files), i+1)

        img = plt.imread(img_files[i])
        plt.tight_layout()
        plt.imshow(img, cmap='gray')
        plt.axis('off')

In [ ]:
files = ['out/100-100.png', 'out/1789-1789.png', 'out/100-1789.png']
plot_pics(files)

## Projecting images to latent space
To find the matching latent vector for a given image file, run:

    python projector.py --outdir=out --target=~/mytargetimg.png \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
    
For optimal results, the target image should be cropped and aligned similar to the FFHQ dataset. The above command saves the projection target out/target.png, result out/proj.png, latent vector out/projected_w.npz, and progression video out/proj.mp4. You can render the resulting latent vector by specifying --projected_w for generate.py:
    
    python generate.py --outdir=out --projected_w=out/projected_w.npz \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl    

### download a picture

In [ ]:
!mkdir -p raw
!wget https://upload.wikimedia.org/wikipedia/commons/6/6d/Shinz%C5%8D_Abe_Official.jpg -O raw/example.jpg

### face alignment

In [ ]:
# face alignment
!python align_images.py raw aligned

In [ ]:
!ls aligned

In [ ]:
files = ['raw/example.jpg', 'aligned/example_01.png']
plot_pics(files)

## Projector

In [ ]:
# projector
!python projector.py --outdir=out --target=aligned/example_01.png \
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl

In [ ]:
from IPython.display import Video
Video('out/proj.mp4')

In [ ]:
!pip uninstall jax jaxlib -y
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
# !git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install ninja

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "./experiments"
NETWORK = "network-snapshot-000120.pkl"
RESUME = os.path.join(EXPERIMENTS, \
                "00003-mainplans1-auto2", NETWORK)
DATA = "../input/mainplans/mainplans1"
SNAP = 35

# Build the command and run it
# cmd = f"python3 ./stylegan2-ada-pytorch/train.py "\
#   f"--snap {SNAP} --resume {RESUME} --mirror=1 --aug=noaug --outdir {EXPERIMENTS} --data {DATA} --gpus=2"

import os
os.system(f"python3 ./stylegan2-ada-pytorch/train.py "\
  f"--snap {SNAP} --resume {RESUME} --mirror=1 --aug=noaug --outdir {EXPERIMENTS} --data {DATA} --gpus=2")

# !{cmd}